In [6]:
!pip install yfinance

In [2]:
!pip install backtesting

In [4]:
pip install --use-pep517 ta

In [5]:
import yfinance as yf
import ta
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

D:\Program Files\Lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [65]:
class SMA_RSI_Strategy(Strategy):
    n1 = 20
    n2 = 50
    rsi_period = 14

    def init(self):
        close = pd.Series(self.data.Close)
        self.sma1 = self.I(ta.trend.sma_indicator, close, self.n1)
        self.sma2 = self.I(ta.trend.sma_indicator, close, self.n2)
        self.rsi = self.I(ta.momentum.rsi, close, self.rsi_period)

    def next(self):
        if crossover(self.sma1, self.sma2) and self.rsi[-1] < 30:
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()

In [66]:
df = yf.download("HINDUNILVR.NS", start = "2024-09-01")
df.columns = df.columns.get_level_values(0)  # <- Add this line
df.dropna(inplace=True)

C:\Users\FARHAN KHAN\AppData\Local\Temp\ipykernel_37416\3736517227.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("HINDUNILVR.NS", start = "2024-09-01")
[*********************100%***********************]  1 of 1 completed


In [67]:
bt = Backtest(df, SMA_RSI_Strategy, cash = 100000, commission = 0.002, exclusive_orders = True)

In [75]:
bt.run()
bt.plot()

Backtest.run:   0%|          | 0/155 [00:00<?, ?bar/s]

GridPlot(id='p2993', ...)

In [74]:
results = bt.run()
results

Backtest.run:   0%|          | 0/155 [00:00<?, ?bar/s]

Start                     2024-09-02 00:00:00
End                       2025-06-27 00:00:00
Duration                    298 days 00:00:00
Exposure Time [%]                    43.90244
Equity Final [$]                 100593.68942
Equity Peak [$]                  109022.80263
Commissions [$]                     805.28824
Return [%]                            0.59369
Buy & Hold Return [%]                 -6.4186
Return (Ann.) [%]                      0.7303
Volatility (Ann.) [%]                12.55779
CAGR [%]                              0.50182
Sharpe Ratio                          0.05816
Sortino Ratio                         0.08811
Calmar Ratio                          0.06425
Alpha [%]                            -1.80587
Beta                                 -0.37385
Max. Drawdown [%]                   -11.36718
Avg. Drawdown [%]                    -4.42373
Max. Drawdown Duration      115 days 00:00:00
Avg. Drawdown Duration       43 days 00:00:00
# Trades                          

In [53]:
!pip install gspread

In [54]:
!pip install oauth2client

In [71]:
def get_or_create_tab(sheet, title, rows=1000, cols=20):
    try:
        return sheet.worksheet(title)
    except gspread.exceptions.WorksheetNotFound:
        return sheet.add_worksheet(title=title, rows=rows, cols=cols)

In [72]:
import gspread
import math
from oauth2client.service_account import ServiceAccountCredentials

# Setup Sheets API
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("glassy-compiler-464222-d7-6e19d9525e88.json", scope)
client = gspread.authorize(creds)

# Open your Google Sheet
sheet = client.open("Trading Logs")

summary = bt.run()
summary_ws = get_or_create_tab(sheet, "Summary")
summary_ws.clear()
summary_ws.append_row(["Metric", "Value"])
for key, value in summary.items():
    summary_ws.append_row([str(key), str(value)])

# Tab 2: Trade Log
log_ws = get_or_create_tab(sheet, "Trades")
log_ws.clear()

trade_df = summary["_trades"]
log_ws.append_row(trade_df.columns.tolist())  # Header row

for row in trade_df.itertuples(index=False):
    # Convert all entries to strings for compatibility
    log_ws.append_row([str(x) for x in row])

# Tab 3: Win Ratio
win_ws = get_or_create_tab(sheet, "WinRatio")
win_ws.clear()

trades_df = summary["_trades"]
win_ratio = (trades_df["ReturnPct"] > 0).mean() * 100

# Check if win_ratio is NaN (which can happen if no trades)
if math.isnan(win_ratio):
    win_ratio = 0.0

win_ws.append_row(["Win Ratio (%)", round(win_ratio, 2)])

Backtest.run:   0%|          | 0/155 [00:00<?, ?bar/s]

{'spreadsheetId': '1DgTrz-2B-KYyrP7Nm__yUFmv6NbF3ShOyeGToz3rSTU',
 'updates': {'spreadsheetId': '1DgTrz-2B-KYyrP7Nm__yUFmv6NbF3ShOyeGToz3rSTU',
  'updatedRange': 'WinRatio!A1:B1',
  'updatedRows': 1,
  'updatedColumns': 2,
  'updatedCells': 2}}

In [73]:

print("Trades:\n", summary["_trades"].head())
print("Win Ratio:", win_ratio)

Trades:
    Size  EntryBar  ExitBar   EntryPrice    ExitPrice    SL    TP          PnL  \
0   -42       111      122  2355.232079  2206.792985  None  None  6234.441980   
1   -47       122      200  2206.792985  2283.399902  None  None -3600.525129   

   ReturnPct  EntryTime   ExitTime Duration   Tag  Entry_sma_indic…(20)  \
0   0.063025 2025-02-07 2025-02-24  17 days  None           2360.934509   
1  -0.034714 2025-02-24 2025-06-23 119 days  None           2327.085474   

   Exit_sma_indic…(20)  Entry_sma_indic…(50)  Exit_sma_indic…(50)  \
0          2327.085474           2364.150200          2335.460889   
1          2324.605286           2335.460889          2329.949346   

   Entry_rsi(14)  Exit_rsi(14)  
0      44.797357     31.238503  
1      31.238503     40.016539  
Win Ratio: 50.0
